In [1]:
import pandas as pd
import numpy as np
from random import shuffle
from sklearn import svm
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import os
import scipy
import sqlite3
from collections import defaultdict

In [2]:
curr_path = os.path.realpath('')
print(1)
conn = sqlite3.connect(curr_path+"/lending-club-loan-data/database.sqlite")
print(2)
# very important, select a random sample of the data for our training set & validation sets 
# 
df = pd.read_sql_query("select * from loan where id in(select id from loan order by random() limit 200000);", conn)
print(3)
df_default = pd.read_sql_query( "select * from loan where loan_status='Charged Off' OR loan_status='Default' OR loan_status='Late (31-120 days)' OR loan_status='Late (16-30 days)' OR loan_status='Does not meet the credit policy. Status:Charged Off' AND id in(select id from loan order by random());", conn)
print(4)
df_fully_paid = pd.read_sql_query("select * from loan where loan_status='Issued' OR loan_status='Fully Paid' OR loan_status='In Grace Period' OR loan_status='Does not meet the credit policy. Status:Fully Paid' AND id in(select id from loan order by random());" , conn)
print(5)
# df_default = pd.read_sql_query("select",conn)
conn.close()
# df = pd.read_sql_query("select * from loan where id in(select id from loan order by random() limit 200000);", conn)
# df_validation = pd.read_sql_query("select * from loan where id in(selec id from loan order by random() limit 100000);", conn)
# print(df)

1
2
3
4
5


In [3]:
# split the data sets into training and validation
# shuffle that dataframe frac = 1 means return all elements sshuffled not a preportion
# grab a 50,000 test set
df = df.sample(frac=1).reset_index(drop=True)
# write the 50,000 test set to a csv file
df_test = pd.read_csv('X_test_set.csv', encoding='ISO-8859-1')

In [48]:
# grab a random set of 25,000 "default" examples
df_default_pick = df_default.sample(n = 50000).reset_index(drop=True)
# grab a random set of 25,000 "fully paid" examples
df_fully_paid_pick = df_fully_paid.sample(n = 50000).reset_index(drop=True)

df_def_1 = df_default_pick.iloc[:int(len(df_default_pick)/2), :]
df_def_2 = df_default_pick.iloc[int(len(df_default_pick)/2):, :]
df_fp_1 = df_fully_paid_pick.iloc[:int(len(df_fully_paid_pick)/2), :]
df_fp_2 = df_fully_paid_pick.iloc[int(len(df_fully_paid_pick)/2):, :]
# append the training sets together first 25,000 defaults, next 25,000 non-defaults
data_train = df_def_1.append(df_fp_1)
data_validation = df_def_2.append(df_fp_2)
# print(len(df_default))
# print(len(df_fully_paid))
# print(len(data_train))
# print(data_train)


In [5]:
# set up 

sub_g_labels = ['A1','A2','A3','A4','A5','B1','B2','B3','B4','B5','C1','C2','C3','C4','C5','D1','D2','D3'\
            ,'D4','D5','E1','E2','E3','E4','E5','F1','F2','F3','F4','F5','G1','G2','G3','G4','G5']

# make sub_grade 
# give sub grade numeric value.
grade_v = [(1.0/35 * i) for i in range(1, 36)]

# dictionary to make sub_grade val to sub_grade string
# sub_grade_dict = dict(zip(sub_g_labels, grade_v))
sub_grade_dict = dict(zip(sub_g_labels, range(1, 36)))

# print(sub_grade_dict)
# print(df.axes)

In [44]:
# iterTuples[0(index)]
ax = ['Index','index','id','member_id','loan_amnt','funded_amnt','funded_amnt_inv','term','int_rate','installment','grade','sub_grade'\
      ,'emp_title','emp_length','home_ownership','annual_inc','verification_status','issue_d','loan_status','pymnt_plan','url','desc','purpose','title','zip_code','addr_state','dti'\
      ,'delinq_2yrs','earliest_cr_line','inq_last_6mths','mths_since_last_delinq','mths_since_last_record','open_acc','pub_rec'\
      ,'revol_bal','revol_util','total_acc','initial_list_status','out_prncp','out_prncp_inv','total_pymnt','total_pymnt_inv'\
      ,'total_rec_prncp','total_rec_int','total_rec_late_fee','recoveries','collection_recovery_fee','last_pymnt_d','last_pymnt_amnt'\
      ,'next_pymnt_d','last_credit_pull_d','collections_12_mths_ex_med','mths_since_last_major_derog','policy_code','application_type'\
      ,'annual_inc_joint','dti_joint','verification_status_joint','acc_now_delinq','tot_coll_amt','tot_cur_bal','open_acc_6m','open_il_6m'\
      ,'open_il_12m','open_il_24m','mths_since_rcnt_il','total_bal_il','il_util','open_rv_12m','open_rv_24m','max_bal_bc'\
      ,'all_util','total_rev_hi_lim','inq_fi','total_cu_tl','inq_last_12m']

index_dict = dict(zip(ax, range(len(ax))))
# 11th spot in the pandas df if read in all columns
sub_grade = 11
loan_amnt = 4
loan_status = 18

def feature(datum):
    # create our simplistic featur vector
    #print(datum)
    s_grade = datum[index_dict['sub_grade']]
    # print(datum[sub_grade])
    return [sub_grade_dict[datum[index_dict['sub_grade']]], datum[index_dict['loan_amnt']]]


default = 0
paid = 0
def prediction_of_y(datum):
    if ((datum[index_dict['loan_status']] == 'Charged Off') or (datum[index_dict['loan_status']] == 'Default') or (datum[index_dict['loan_status']] == 'Late (31-120 days)' ) or (datum[index_dict['loan_status']] == 'Late (16-30 days)') or (datum[index_dict['loan_status']] =='Does not meet the credit policy. Status:Charged Off')):
        global default 
        default += 1
        return True
    else:
        global paid 
        paid += 1
        return False

# X_train = [feature(d) for d in data_train]
X_train = []
pp = True
for row in data_train.itertuples():
    if pp: 
        # print(row)
        pp = False
    X_train.append(feature(row))
    
# y_train = [prediction_of_y(d) for d in data_train]
y_train = []
for row in data_train.itertuples():
    y_train.append(prediction_of_y(row))
    

In [45]:
l_status_dict = defaultdict(int)
b = True
for row in df_default.itertuples():
    if b:
        print(len(row))
        # for i in range(len(row)):
            # print(row[74])
            #print(str(i)+' row[i] is:  '+str()
            #print(str(i)+'  ax[i] is:  '+str(ax[i]))
        # print(len(row))
        b = False
    # print(row[3])
    if row[loan_status] not in l_status_dict:
        l_status_dict[row[loan_status]] += 1
        
print(l_status_dict.keys())
print(default)
print(paid)
print(len(ax))
# print(df.keys)

76
dict_keys(['Charged Off', 'Default', 'Late (31-120 days)', 'Late (16-30 days)', 'Does not meet the credit policy. Status:Charged Off'])
25000
25000
76


In [37]:
# a = list(zip(X_train[:50],y_train[:50]))
# print(a)
# pipline to scale the feature vectors and run the svm
rbf_svm_clf = Pipeline((
        ('scaler', StandardScaler()),
        ("rbf_svc", SVC(kernel='rbf', C=1))
    ))

linear_svm_clf = Pipeline((
        ('scaler',StandardScaler()),
        ('linear_svc', LinearSVC(C=1))
    ))


In [46]:
# clf = svm.SVC(C=1, kernel='rbf')
# clf.fit(scaler.transform(X_train), y_train)
rbf_svm_clf.fit(X_train, y_train)
train_predictions = rbf_svm_clf.predict(X_train)

# test_predictions = clf.predict(X_test)
# print(train_predictions)

In [38]:
# rbf_svm_clf_C1000.fit(X_train, y_train)
# train_predictions_C1000 = rbf_svm_clf_C1000.predict(X_train)
linear_svm_clf.fit(X_train, y_train)
train_lin_predict = linear_svm_clf.predict(X_train)
print(find_results(train_lin_predict,y_train))

0.62288


In [47]:
# what % acc did we get?

def find_results(tp,y_t):
    
    results = list(zip(tp,y_t))
    correct = 0

    for i in range(len(results)):
        if results[i][0] == results[i][1]:
            correct +=1
    
    return(correct/float(len(results)))

print(find_results(train_predictions, y_train))
# print(find_results(train_predictions_C1000, y_train))
# print(find_results(train_predictions_C001, y_train))

0.6273


In [29]:
# prep the SVM to run on the validation set
X_valid = []
for row in data_validation.itertuples():
    X_valid.append(feature(row))
    
y_valid = []
for row in data_validation.itertuples():
    y_valid.append(prediction_of_y(row))

    
#find the acc of the validation set
valid_predictions = rbf_svm_clf.predict(X_valid)

results_valid = list(zip(valid_predictions, y_valid))
correct_valid = 0
flag = True
for i in range(len(results_valid)):
    if flag:
        print(results_valid[i][0])
        print(results_valid[i][1])
        flag = False
    if results_valid[i][0] == results_valid[i][1]:
        correct_valid +=1

print(correct_valid)
print(correct_valid/float(len(results_valid)))


True
True
31284
0.62568
